# Lab2 - PuLP Library

<b>Information on group members:</b><br>
1) 154974, Andrii Chmutov <br>
2) 158669, Vasyl Korzavatykh

In [1]:
from pulp import (
    LpAffineExpression,
    LpConstraint,
    LpMaximize,
    LpProblem,
    LpStatus,
    LpVariable,
    lpSum,
)

## Introduction - brief tutorial on PuLP

In [2]:
# Create an LpProblem instance; LpMaximize = objective function is to be maximized
model = LpProblem(name="some-problem", sense=LpMaximize)

In [3]:
# Initialize the decision variables. We can set the name and lower bound as well.
# To create an array of variables, you can use comprehensions of LpProblem.dicts.

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

In [4]:
# An example expression
expression = 3 * x1 + 2 * x2
type(expression)

pulp.pulp.LpAffineExpression

In [5]:
# An example constraint
constraint = 2 * x1 + 3 * x2 >= 5
type(constraint)

pulp.pulp.LpConstraint

Ok, let's now use PuLP to solve the below problem:
$max$ $4x_1 + 2x_2$ <br>
s.t.<br>
     $1x_1 + 1x_2 \geq 1$ <br>
     $2x_1 + 1x_2 \leq 6$ <br>
     $-1x_1 + x_2 = 1$ <br>
     $x_1 \geq 0$ <br>
     $x_2 \geq 0$ 

In [6]:
# Problem
model = LpProblem(name="test-problem", sense=LpMaximize)

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

model += (1 * x1 + 1 * x2 >= 1, "#1 constraint")
model += (2 * x1 + 1 * x2 <= 6, "#2 constraint")
model += (-1 * x1 + 1 * x2 == 1, "#3 constraint")

# Objective function
obj_func = 4 * x1 + 2 * x2
model += obj_func

model

test-problem:
MAXIMIZE
4*x1 + 2*x2 + 0
SUBJECT TO
#1_constraint: x1 + x2 >= 1

#2_constraint: 2 x1 + x2 <= 6

#3_constraint: - x1 + x2 = 1

VARIABLES
x1 Continuous
x2 Continuous

In [7]:
# Solve the problem
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/warlock/prog/venvs/tf/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/fde40ed4611f4e14bac4b442bc340300-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/fde40ed4611f4e14bac4b442bc340300-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-6) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 12
After Postsolve, objective 12, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 12 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00


In [8]:
# Print status
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [9]:
# Print objective value
print(f"objective: {model.objective.value()}")

objective: 12.000000199999999


In [10]:
# Print constraint values
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

#1_constraint: 3.3333334
#2_constraint: 9.999999983634211e-08
#3_constraint: 0.0


In [11]:
model.variables()

[x1, x2]

In [12]:
print(x1.value())
print(x2.value())

1.6666667
2.6666667


### The same code but using dictionaries and other nice tricks


In [13]:
model = LpProblem(name="some-problem", sense=LpMaximize)

In [14]:
var_names = ["First", "Second"]
x = LpVariable.dicts("x", var_names, 0)
x

{'First': x_First, 'Second': x_Second}

In [15]:
const_names = ["GE", "LE", "EQ"]
sense = [1, -1, 0]  # GE, LE, EQ
coefs = [[1, 1], [2, 1], [-1, 1]]  # Matrix coefs
rhs = [1, 6, 1]

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(2)])
    model += LpConstraint(e=expr, sense=s, name=cn, rhs=r)

obj_coefs = [4, 2]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(2)])

model

some-problem:
MAXIMIZE
4*x_First + 2*x_Second + 0
SUBJECT TO
GE: x_First + x_Second >= 1

LE: 2 x_First + x_Second <= 6

EQ: - x_First + x_Second = 1

VARIABLES
x_First Continuous
x_Second Continuous

In [16]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for n in var_names:
    print(x[n].value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/warlock/prog/venvs/tf/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/74823b6da19a452290d32f866ac53980-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/74823b6da19a452290d32f866ac53980-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-6) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 12
After Postsolve, objective 12, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 12 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00


# Homework - use the PuLP library to solve the following OR problem 

Johnson & Sons company manufactures 6 types of toys. Each toy is produced by utilizing at least one Machine 1-4, requiring a different production time (see Table below). For instance, product A requires 4 minutes on Machine 1, 4 minutes on Machine 2, 0 Minutes on Machine 3, and 0 minutes on Machine 4 (all machines must be utilized to produce a toy unless the production time equals 0). Each machine is available for a different number of hours per week. The company aims to identify the number (product-mix) of toys that must be manufactured to maximize the income (can be continuous). Notably, each toy can be sold for a different price. Due to market expectations, the company wants to manufacture twice as many F toys as A. Furthermore, the number of toys B should equal C. Solve this problem using the PuLP library. Prepare a report (in the jupyter notebook) containing information on the following:
<ol>
<li>The number of toys to manufacture;</li>
<li>The expected income;</li>
<li>The production time required on each machine;</li>
</ol>
Consider the total and partial values, i.e., considered for each toy A-F separately (e.g., income resulting from selling toy A). Also, answer the following questions concerning the found solution:
<ol>
<li>Which toy(s) production should be focused on?  </li>
<li>Is there a toy that can be excluded from consideration for production? </li>
<li>Is there a machine that is not fully utilized?</li>
</ol>

| Toy | Machine 1 | Machine 2 | Machine 3 | Machine 4 | Selling price |
| --- | --- | --- | --- | --- | --- |
| A | 4 (minutes!) | 4 | 0 | 0 | 2.50 USD |
| B | 0 | 3 | 3 | 0 | 1.00 USD |
| C | 5 | 0 | 2 | 5 | 4.00 USD |
| D | 2 | 4 | 0 | 4 | 3.00 USD |
| E | 0 | 4 | 5 | 2 | 3.50 USD |
| F | 2 | 2 | 1 | 1 | 4.00 USD |
| Production time limit (hours!) | 120 | 60 |  80 |  120 |  |

## Data

In [17]:
import pandas as pd


def machine_data() -> tuple[pd.DataFrame, pd.Series, pd.Series]:
    names = list("ABCDEF")
    machines = [f"Machine_{i}" for i in range(4)]
    coefficients = [
        [4, 4, 0, 0],
        [0, 3, 3, 0],
        [5, 0, 2, 5],
        [2, 4, 0, 4],
        [0, 4, 5, 2],
        [2, 2, 1, 1],
    ]
    return (
        pd.DataFrame(
            coefficients,
            columns=machines,
            index=pd.Index(names, name="Toy"),
            dtype=float,
        ),
        pd.Series(
            [120, 60, 80, 120],
            index=pd.Index(machines, name="Machine"),
            dtype=float,
            name="Production time limit",
        )
        * 60,
        pd.Series(
            [2.5, 1.0, 4.0, 3.0, 3.5, 4.0],
            index=pd.Index(names, name="Toy"),
            dtype=float,
            name="Selling price",
        ),
    )


def market_data() -> tuple[pd.DataFrame, pd.Series]:
    names = list("ABCDEF")
    constraint_names = [f"Constraint_{i}" for i in range(2)]
    coefficients = [
        [1, 0],
        [0, -1],
        [0, 1],
        [0, 0],
        [0, 0],
        [-2, 0],
    ]
    return (
        pd.DataFrame(
            coefficients,
            columns=constraint_names,
            index=pd.Index(names, name="Toy"),
            dtype=float,
        ),
        pd.Series(
            [0] * len(constraint_names),
            index=pd.Index(constraint_names, name="Constraint"),
            dtype=float,
            name="Ratio",
        ),
    )

## Processing

In [18]:
import pandas as pd


def denormalize_name(name: str) -> str:
    return name.replace("_", " ")


def denormalize_name_series(series: pd.Series) -> pd.Series:
    series = series.copy()
    series.name = denormalize_name(series.name)
    series.index = [denormalize_name(x) for x in rhs_time.index]
    return series

## Modeling

In [19]:
def make_variables(df: pd.DataFrame, prefix: str = "x") -> pd.Series:
    return pd.Series(LpVariable.dicts(prefix, df.index.tolist(), 0))


def make_constraints(
    df: pd.DataFrame, rhs: pd.Series, variables: pd.Series, sense: int
) -> list[LpAffineExpression]:
    assert df.shape[0] == variables.shape[0]
    assert df.shape[1] == rhs.shape[0]

    return [
        LpConstraint(
            e=lpSum([
                variables[var_name] * df.loc[var_name, constr_name]
                for var_name in df.index
            ]),
            sense=sense,
            name=constr_name,
            rhs=rhs[constr_name],
        )
        for constr_name in rhs.index
    ]


def make_objective(
    variables: pd.Series, objective_coefficients: pd.Series
) -> LpAffineExpression:
    assert variables.shape[0] == objective_coefficients.shape[0]
    assert frozenset(variables.index) == frozenset(objective_coefficients.index)

    return lpSum([
        variables[var_name] * objective_coefficients[var_name]
        for var_name in variables.index
    ])


def make_model(
    constraints: list[LpAffineExpression],
    objective_function: LpAffineExpression,
    name: str,
    sense: int = LpMaximize,
) -> LpProblem:
    model = LpProblem(name=name, sense=sense)

    for constraint in constraints:
        model += constraint
    model += objective_function

    return model


def make_model_facade(
    df_machines: pd.DataFrame,
    df_market: pd.DataFrame,
    rhs_time: pd.Series,
    rhs_market: pd.Series,
    obj_price: pd.Series,
    name: str,
) -> LpProblem:
    # Constraints
    variables = make_variables(df_market)
    constraints = make_constraints(
        df_machines, rhs_time, variables, -1
    ) + make_constraints(df_market, rhs_market, variables, 0)

    # Objective_function
    objective_function = make_objective(variables, obj_price)

    # Modeling
    return make_model(constraints, objective_function, name=name, sense=LpMaximize)

## Solution

In [27]:
# Processing
df_machines, rhs_time, obj_price = machine_data()
df_market, rhs_market = market_data()

# Modeling
model = make_model_facade(
    df_machines, df_market, rhs_time, rhs_market, obj_price, "Johnsons&Sons"
)
_ = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/warlock/prog/venvs/tf/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b2fb3aeb1a6d46649daf642abab584a6-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/b2fb3aeb1a6d46649daf642abab584a6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 39 RHS
At line 46 BOUNDS
At line 47 ENDATA
Problem MODEL has 6 rows, 6 columns and 21 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-2) rows, 4 (-2) columns and 14 (-7) elements
0  Obj -0 Dual inf 6.6916741 (4)
2  Obj 5412.766
Optimal - objective value 5412.766
After Postsolve, objective 5412.766, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 5412.765957 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclo

## Report

### Production and income prediction

#### 1. The number of toys to manufacture

In [21]:
def make_toys_count(model: LpProblem) -> pd.Series:
    index = [str(x).removeprefix("x_") for x in model.variables()]
    return pd.Series(map(LpVariable.value, model.variables()), index=index, name="Count")


# Add footer
pd.concat([
    make_toys_count(model),
    pd.Series([make_toys_count(model).sum()], index=["SUM"], name="Count"),
]).to_frame()

,Count
A,153.191490
B,944.680850
C,944.680850
D,0.000000
E,0.000000
F,76.595745
SUM,2119.148935


#### 2. Expected income
The expected value of the income is approximately equal to $5412.77$\$


In [22]:
model.objective.value()

5412.765955


#### 3. The production time required on each machine
Here are presented busy hours of each of the machine per week.


In [23]:
import numpy as np


def make_hours(
    model: LpProblem, df_machines: pd.Series, rhs_time: pd.Series
) -> pd.Series:
    hours = (
        np.multiply(
            df_machines.to_numpy(),
            np.expand_dims(make_toys_count(model).to_numpy(), axis=1),
        )
        .sum(axis=0)
        .__truediv__(60)
        .round(2)
    )

    denormalized = denormalize_name_series(rhs_time)
    return pd.Series(hours, name="Hours", index=denormalized.index)


make_hours(model, df_machines, rhs_time).to_frame()

,Hours
Machine 0,91.49
Machine 1,60.00
Machine 2,80.00
Machine 3,80.00


### Analysis

#### 1. Which toy(s) production should be focused on?
Based on the data presented below the cell, Toy C brings the most profit (approximately $3778.72$\$), so, logically, company should focus on its production.

In [24]:
(
    (make_toys_count(model) * obj_price)
    .rename("Income $")
    .round(2)
    .sort_values(ascending=False)
    .to_frame()
)

,Income $
C,3778.72
B,944.68
A,382.98
F,306.38
D,0.00
E,0.00



#### 2. Is there a toy that can be excluded from consideration for production?

As we've already seen above, the optimal solution implies zero production of Toys D and E, so the Toy D and the Toy E can be excluded from consideration for production. To test that we would run the same model but without Toys D and E and the constraints they provide. The numbers for produced toys should stay the same.


In [25]:
restricted_model = make_model_facade(
    df_machines.drop(["D", "E"]),
    df_market.drop(["D", "E"]),
    rhs_time,
    rhs_market,
    restricted_obj := obj_price.drop(["D", "E"]),
    "restricted-Johnsons&Sons",
)
restricted_model.solve()

(
    (make_toys_count(restricted_model) * restricted_obj)
    .rename("Income $")
    .round(2)
    .sort_values(ascending=False)
    .to_frame()
)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/warlock/prog/venvs/tf/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e527d92c6564497384f32aa287f2eb51-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/e527d92c6564497384f32aa287f2eb51-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 31 RHS
At line 38 BOUNDS
At line 39 ENDATA
Problem MODEL has 6 rows, 4 columns and 15 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-3) rows, 2 (-2) columns and 6 (-9) elements
0  Obj -0 Dual inf 13.999998 (2)
2  Obj 5412.766
Optimal - objective value 5412.766
After Postsolve, objective 5412.766, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 5412.765957 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallcloc

,Income $
C,3778.72
B,944.68
A,382.98
F,306.38


#### 3. Is there a machine that is not fully utilized?
As we can see from the data below, Machine 0 and Machine 3 were not used to maximal extend.


In [26]:
pd.concat(
    [
        denormalized := denormalize_name_series(rhs_time) / 60,
        made := make_hours(model, df_machines, rhs_time).rename("Hours used"),
        pd.Series(denormalized - made, name="Hours left"),
        pd.Series(made / denormalized, name="Utilization"),
    ],
    axis=1,
).round(2)

,Production time limit,Hours used,Hours left,Utilization
Machine 0,120.0,91.49,28.51,0.76
Machine 1,60.0,60.00,0.00,1.00
Machine 2,80.0,80.00,0.00,1.00
Machine 3,120.0,80.00,40.00,0.67
